In [146]:
import geopandas as gpd
import pandas as pd
#from gerrychain import Graph, Partition, Election
#from gerrychain.updaters import Tally, cut_edges


# vtd['vote_difference'] = vtd.EL16G_GV_R - vtd.EL16G_GV_D
# vtd.plot("vote_difference", legend = True, figsize = (20,20) )
# vtd_county.plot("EL12G_GV_R", legend = True, figsize = (10,10) )
#vtd_county = vtd.dissolve(by='County', aggfunc='sum')
#vtd_county.to_file("./data/NC_county_merged.json", driver="GeoJSON")  
#.to_crs("EPSG:4269")


In [179]:
# vtd = gpd.read_file("./data/NC_VTD.shp").to_crs("ESRI:103500")
# bg = gpd.read_file("./data/tl_2016_37_bg.shp").to_crs("ESRI:103500")
# blocks = gpd.read_file("./data/blocks/tl_2016_37_tabblock10.shp").to_crs("ESRI:103500")

In [57]:
vtd = gpd.read_file("./data/NC_VTD.shp").to_crs("EPSG:4269")
bg = gpd.read_file("./data/tl_2016_37_bg.shp").to_crs("EPSG:4269")
blocks = gpd.read_file("./data/blocks/tl_2016_37_tabblock10.shp").to_crs("EPSG:4269")

In [ ]:
joined_within = gpd.sjoin(blocks, vtd, how = 'left', predicate = 'within' )
joined_within[["GEOID10", "VTD"]].groupby("GEOID10").agg('count').value_counts()

In [126]:
# geo_table = blocks
# geo_id = "GEOID10"
i = 0
geo_table = [ blocks, bg][i]
geo_id = ["GEOID10", "GEOID"][i]

joined = gpd.sjoin(geo_table, vtd, how = 'left',  predicate = 'intersects' )
mapping = joined[[geo_id, "VTD"]]

display(joined[[geo_id, "VTD"]].groupby(geo_id).agg('count').value_counts())



VTD
1      201318
2       73931
3       12086
4        1524
5         112
6          13
7           3
dtype: int64

In [178]:
#counts = joined[[geo_id, "VTD"]].groupby(geo_id).agg('count').rename(columns = {"VTD" : "NUM INTERSECTIONS"})
#m = joined.merge(counts, left_on = geo_id, right_index = True)
#m[m["NUM INTERSECTIONS"] <= 5].plot("NUM INTERSECTIONS", legend = True)

In [176]:
# Intersect geos with VTDs, compute area ratios
geo_table_with_map = geo_table.merge(mapping)#.merge(vtd[["VTD", "geometry"]], suffixes = ('_block', '_vt') )
geo_with_vtd = geo_table_with_map.merge(vtd,
                                                      left_on = "VTD", right_on = "VTD",
                                                      suffixes = ('_geo', '_vtd'))

geo_geos = gpd.GeoSeries(geo_with_vtd.geometry_geo).to_crs("ESRI:103500")


In [177]:
geo_with_vtd['geo_intersect'] = geo_geos.intersection(gpd.GeoSeries(geo_with_vtd.geometry_vtd).to_crs("ESRI:103500"))

geo_with_vtd['geo_area'] = geo_geos.area
geo_with_vtd['intersect_area'] = gpd.GeoSeries(geo_with_vtd.geo_intersect).to_crs("ESRI:103500").area
geo_with_vtd['area_proportion'] =  geo_with_vtd['intersect_area'] / geo_with_vtd['geo_area']

In [138]:
geo_with_vtd[geo_with_vtd.area_proportion >= .2][[geo_id, "VTD"]].groupby(geo_id).agg('count').value_counts()

VTD
1      288215
2         772
dtype: int64

In [143]:
#geo_with_vtd.rename(columns = {'geometry_geo' : "geometry" } ).plot("area_proportion", legend = True)

# Adding in population level blocks

In [147]:
pops = pd.read_csv('./data/us2016_block_pops.csv')

In [152]:
pops['block_fips'] = pops.block_fips.astype("str")

In [180]:
geo_with_pop = geo_with_vtd.merge(pops, left_on = geo_id, right_on = 'block_fips',)

In [190]:
geo_with_pop.drop(columns = ["geometry_geo", "geometry_vtd", "geo_intersect"]).to_csv("geo_with_pop.csv")

In [204]:
vtd.columns

Index(['ALAND10', 'AWATER10', 'VTD', 'County', 'VTD_Key', 'VTD_Name',
       'PL10AA_TOT', 'PL10VA_TOT', 'EL08G_GV_D', 'EL08G_GV_R', 'EL08G_GV_L',
       'EL08G_GV_T', 'EL08G_USS_', 'EL08G_US_1', 'EL08G_US_2', 'EL08G_US_3',
       'EL08G_US_4', 'EL10G_USS_', 'EL10G_US_1', 'EL10G_US_2', 'EL10G_US_3',
       'EL10G_US_4', 'EL12G_GV_D', 'EL12G_GV_R', 'EL12G_GV_L', 'EL12G_GV_W',
       'EL12G_GV_1', 'EL12G_GV_T', 'EL14G_USS_', 'EL14G_US_1', 'EL14G_US_2',
       'EL14G_US_3', 'EL14G_US_4', 'Shape_Leng', 'Shape_Area', 'EL12G_PR_D',
       'EL12G_PR_R', 'EL12G_PR_L', 'EL12G_PR_W', 'EL12G_PR_1', 'EL12G_PR_T',
       'EL16G_PR_R', 'EL16G_PR_D', 'EL16G_PR_L', 'EL16G_PR_W', 'EL16G_PR_T',
       'EL16G_USS_', 'EL16G_US_1', 'EL16G_US_2', 'EL16G_US_3', 'EL16G_GV_D',
       'EL16G_GV_R', 'EL16G_GV_L', 'EL16G_GV_T', 'BPOP', 'nBPOP', 'judge',
       'newplan', 'oldplan', 'TOTPOP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN',
       'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'HISP', 'H_WHITE',
       'H_BLACK

In [195]:
geo_with_pop['pop_proportion'] = geo_with_pop.pop2016 /  geo_with_pop.TOTPOP

In [201]:
# This filtering is necessary, since otherwise we end up with blocks that have more population than the VTD they represent.
geo_with_pop[geo_with_pop.area_proportion >= .1].pop_proportion.sort_values()

127114    0.000000
112671    0.000000
274810    0.000000
112668    0.000000
216513    0.000000
            ...   
208233    0.733688
19681     0.789588
312766    0.796923
255614    0.804348
55260     0.935275
Name: pop_proportion, Length: 290551, dtype: float64

In [ ]:
geo_with_pop['pop_proportion']